In [6]:
import open3d as o3d
path = "filtered.pcd"
pcd = o3d.io.read_point_cloud(path)
o3d.visualization.draw_geometries([pcd])
alpha = 0.03
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

alpha=0.030


In [9]:
import numpy as np
tetra_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)
for alpha in np.logspace(np.log10(0.5), np.log10(0.01), num=6):
    print(f"alpha={alpha:.3f}")
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
        pcd, alpha, tetra_mesh, pt_map)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

alpha=0.500
alpha=0.229
alpha=0.105
alpha=0.048
alpha=0.022
alpha=0.010


In [10]:
gt_mesh = o3d.io.read_point_cloud(path)
gt_mesh.compute_vertex_normals()
pcd = gt_mesh.sample_points_poisson_disk(3000)
o3d.visualization.draw_geometries([pcd])

AttributeError: 'open3d.cpu.pybind.geometry.PointCloud' object has no attribute 'compute_vertex_normals'

In [12]:
pcd = o3d.io.read_point_cloud(path)
print(pcd)
o3d.visualization.draw_geometries([pcd])



PointCloud with 172285 points.


In [13]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh])

run Poisson surface reconstruction


RuntimeError: [1;31m[Open3D Error] (static std::tuple<std::shared_ptr<open3d::geometry::TriangleMesh>, std::vector<double, std::allocator<double> > > open3d::geometry::TriangleMesh::CreateFromPointCloudPoisson(const open3d::geometry::PointCloud&, size_t, float, float, bool, int)) /root/Open3D/cpp/open3d/geometry/SurfaceReconstructionPoisson.cpp:732: Point cloud has no normals
[0;m

In [14]:
pcd = pcd = o3d.io.read_point_cloud(path)
pcd.normals = o3d.utility.Vector3dVector(np.zeros(
    (1, 3)))  # invalidate existing normals

pcd.estimate_normals()
o3d.visualization.draw_geometries([pcd], point_show_normal=True)


In [15]:
pcd.orient_normals_consistent_tangent_plane(100)
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

In [16]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 172285 / 167452
[Open3D DEBUG] #   Got kernel density: 0.12786602973937988 (s), 791.5390625 (MB) / 791.5390625 (MB) / 1224 (MB)
[Open3D DEBUG] #     Got normal field: 0.2904388904571533 (s), 827.9140625 (MB) / 827.9140625 (MB) / 1224 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.399197e-05 / 5.856306e+00
[Open3D DEBUG] #       Finalized tree: 0.2989051342010498 (s), 830.4140625 (MB) / 830.4140625 (MB) / 1224 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.4283120632171631 (s), 830.5390625 (MB) / 830.5390625 (MB) / 1224 (MB)
[Open3D DEBUG] #Set point constraints: 0.1298360824584961 (s), 830.5390625 (MB) / 830.5390625 (MB) / 1224 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 2220751 / 1338104 / 1199897
[Open3D DEBUG] Memory Usage: 830.539 MB
[Open3D DEBUG] # Linear system solved: 0.7757508754730225 (s), 830.5390625 (MB) / 830.5390625 (MB) / 1224 (MB)
[Open3D DEBUG] Got average: 0.036604881286621

In [20]:
import matplotlib as plt
import matplotlib.cm as plt

densities = np.asarray(densities)
density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
o3d.visualization.draw_geometries([density_mesh])

/tmp/ipykernel_23075/75420362.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  density_colors = plt.get_cmap('plasma')(


In [21]:
print('remove low density vertices')
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)
print(mesh)
o3d.visualization.draw_geometries([mesh])

remove low density vertices
TriangleMesh with 496790 points and 1007212 triangles.
